In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 필사 대상 노트북 원본
https://www.kaggle.com/jsaguiar/lightgbm-7th-place-solution
# 대회 주제
고객의 대출상환 능력을 가늠하자

# 평가지표
Area under the ROC curve

# 목표
TARGET 이라는 변수에 상환 확률을 주어야 한다.

In [ ]:
import os
import gc # 나중에 메모리 관리 해야 함
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 진행방식
기존 코드가 함수형으로 되어 있어서 진행 순서대로 풀어서 다시 쓴다.

In [ ]:
# main 상단에 있던 일반 전역변수들
# GENERAL CONFIGURATIONS
NUM_THREADS = 4
DATA_DIRECTORY = "../input/"
SUBMISSION_SUFIX = "_model2_04"

In [ ]:
# 출력 크기 제한
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', 100)

In [ ]:
# 기본적으로 debug=False였지만 main함수 실행조건이 debug=True라서 True로 변환해서 쭉 사용한다
debug = True

In [ ]:
# 데이터 갯수 제한 하기 위함  
num_rows = 30000 if debug else None 

In [ ]:
# 이제 아래 코드를 사용해야 하지만 get_train_test 메소드는 풀어서 사용하기 때문에 주석처리함
# df = get_train_test(DATA_DIRECTORY, num_rows= num_rows)

In [ ]:
NUM_THREADS = 4
DATA_DIRECTORY = '../input/home-credit-default-risk/'
SUBMISSION_SUFIX = '_model2_04'

In [ ]:
path = DATA_DIRECTORY
num_rows = num_rows

In [ ]:
train = pd.read_csv(os.path.join(path, 'application_train.csv'), nrows= num_rows)
test = pd.read_csv(os.path.join(path, 'application_test.csv'), nrows= num_rows)
df = train.append(test) # train,test 합친다
# train,test각각 데이터 3만개가 있으면 된다
print(train.info())
print(test.info())

In [ ]:
# 타겟 컬럼의 imbalance를 확인하기 위함
# 1이 상당히 적다는걸 인지하고 진행
train['TARGET'].value_counts().plot.bar()

In [ ]:
# 사용하지 않는 메모리 회수
gc.collect()

# Preprocessing
주로 대출 상환 능력을 평가하기위한 피쳐를 만든다

In [ ]:
# Data Cleansing
# 전체 데이터에서 CODE_GENDER 피쳐에서 XNA가 있는 데이터는 4개 뿐이라 제거
df = df[df['CODE_GENDER'] != 'XNA']

In [ ]:
# 사용할 데이터셋의 사람들의 수입에 제한에 두기 위해
df = df[df['AMT_INCOME_TOTAL'] < 20000000]

In [ ]:
# DAYS_EMPLOYED 사람들의 고용된 날짜
# 해당 피쳐에서 이상값으로 365243이 있는데 해당 값을 지우기 위해서
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

In [ ]:
# 해드폰 교체 시기인데 0인 값들은 제거
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [ ]:
# Flag_document features - count and kurtosis
# FLAG_DOC이 있는 피쳐들은 거의 binary라서 합친다
docs = [f for f in df.columns if 'FLAG_DOC' in f]
# 0,1,2,3 중에 1인것들의 갯수가 월등히 많아서 1인것의 갯수
df['DOCUMENT_COUNT'] = df[docs].sum(axis=1)
# kurtosis가 있는 것들을 별도 피쳐로 생성
df['NEW_DOC_KURT'] = df[docs].kurtosis(axis=1)


In [ ]:
# Categorical age - based on target=1 plot
# DAYS_BIRTH는 살아온 날
# DAYS_BIRTH를 카테고리 피쳐로 만들어 AGE_RANGE로 저장해 새로운 피쳐로 생

def get_age_label(days_birth):
    """ Return the age group label (int). 
        나이를 나이대로 분류
    """
    age_years = -days_birth / 365
    if age_years < 27: return 1
    elif age_years < 40: return 2
    elif age_years < 50: return 3
    elif age_years < 65: return 4
    elif age_years < 99: return 5
    else: return 0
    
df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x: get_age_label(x))

In [ ]:
# New features based on External sources
# EXT_SOURCES 3가지는 항상 feature importance와 correlation에서 1,2,3등으로 나와서 묶어서 곱해 하나의 피쳐로 만든다
# 현업의 경우 설명을 해주어야 하지만 어렵다. 하지만 캐글에선 어떻게든 성능을 올리는게 목표기 떄문에 그냥 쓴다
df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
# 가중치를 주어서 합하기
# 가중치 크기는 다른 사람들의 커널의 피쳐분석에서 항상 높은 순위로 나온대로 사용
df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3
# 각 수리통계 수치의 이름을 피쳐이름에 넣으면서 계산해 내기 위해 for문으로 진행
np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
    feature_name = 'EXT_SOURCES_{}'.format(function_name.upper()) # 피쳐이름 생성
    # 피쳐이름을 eval안의 코드에 적용해서 실행
    df[feature_name] = eval('np.{}'.format(function_name))(
        df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1) 

In [ ]:
# Credit ratios
# 사람의 재정상태를 평가하기 위해서 2가지 피쳐를 생성
# 사람들의 빚과 연금간의 비율 피쳐 생성
df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
# 빚과 대출 발생시의 물가 비율 피쳐 생성
df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

In [ ]:
# Income ratios
# 연금과 수입 비율
df['ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
# 빚과 수입 비율, 수입과 지출에 대한 평갸
df['CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
# 수입과 노동일수 비율, 돈버는 능력 평가
df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
# 나이와 수입 비율
df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']

In [ ]:
# Time ratios
# 노동일수와 나이 비율
df['EMPLOYED_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
# 고객정보가 수정된 날짜와 나이 비율
df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
# 보유차량 연식과 나이 비율
df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
# 보유차량 연식과
df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

# 영상2번

In [ ]:
group = ['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']
df[group]

In [ ]:
# Groupby: Statistics for applications in the same group
# 통계 연산을 특정 피쳐들 기준으로 연산하여 새로운 피쳐를 만든다

# 아래 4가지 메소드는 기본통계 연산을 하는 메소드
# df는 데이터
# group_cols는 선택한 피쳐목록
# counted는 계산대상 피쳐
# agg_name은 새로 만들어진 피쳐의 이름
def do_mean(df, group_cols, counted, agg_name):
    # group_cols기즌으로 groupby후 연산
    # reset_index로 정리 후 rename으로 기존 피쳐 이름이 counted인 것을 agg_name으로 변경
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(
        columns={counted: agg_name})
    # df와 gp를 붙이는데 on에 지정된 피쳐들을 기준으로 왼쪽(left)에 붙인
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


def do_median(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


def do_std(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].std().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


def do_sum(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

# 아래 group에 명시된 피쳐들 기준으로 통계연산을 해서 새로운 피쳐를 만든다
group = ['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']
df = do_median(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEDIAN')
df = do_std(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_STD')
df = do_mean(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_MEAN')
df = do_std(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_STD')
df = do_mean(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_MEAN')
df = do_std(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_STD')
df = do_mean(df, group, 'AMT_CREDIT', 'GROUP_CREDIT_MEAN')
df = do_mean(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_MEAN')
df = do_std(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_STD')

In [ ]:
# Encode categorical features (LabelEncoder)
# 카테고리 피쳐들 인코딩
def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers (0,1,2,3...) with pandas.factorize. """
    # scikit의 라벨인코더보단 pandas.factorize가 더 빨르고 데이터가 많아서 factorize를 사용하는것 같다    
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

def drop_application_columns(df):
    """ Drop features based on permutation feature importance. """
    # permutation feature importance에 기반해서 피쳐를 제거한다
    drop_list = [
        'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START',
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'REG_CITY_NOT_WORK_CITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
        'COMMONAREA_MODE', 'NONLIVINGAREA_MODE', 'ELEVATORS_MODE', 'NONLIVINGAREA_AVG',
        'FLOORSMIN_MEDI', 'LANDAREA_MODE', 'NONLIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MODE',
        'FLOORSMIN_AVG', 'LANDAREA_AVG', 'FLOORSMIN_MODE', 'LANDAREA_MEDI',
        'COMMONAREA_MEDI', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE', 'NONLIVINGAPARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
        'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'YEARS_BUILD_MEDI', 'ENTRANCES_MODE',
        'NONLIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'LIVINGAPARTMENTS_MEDI',
        'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_MEDI', 'LIVINGAREA_MEDI',
        'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_AVG', 'HOUSETYPE_MODE',
        'FONDKAPREMONT_MODE', 'EMERGENCYSTATE_MODE'
    ]
    # Drop most flag document columns
    for doc_num in [2,4,5,6,7,9,10,11,12,13,14,15,16,17,19,20,21]:
        drop_list.append('FLAG_DOCUMENT_{}'.format(doc_num))
    df.drop(drop_list, axis=1, inplace=True)
    return df


df, le_encoded_cols = label_encoder(df, None)
df = drop_application_columns(df)

## permutaiton feature importance
보통 feature importance는 여러번 피쳐를 바꾸어 가며 학습해 최고의 성능을 주는 피쳐를 선택한다.  
그러나 permuation은 랜덤하게 피쳐를 선택해서 단 한번만 학습을 시킨다.
- 모든 피쳐를 랜덤하게 섞어서 피쳐와 타겟관의 연결고리를 끊는다
- 학습 후 예측값과 실제값의 차이를 계상해서 피쳐의 영향력을 파악한다
- 만약 피쳐가 타겟에 영향이 크면 예측 정확도가 크게 감소하기 때문에 해당 피쳐는 중요도가 크다고 할 수 있다.
- 랜덤하기 때문에 매번 결과가 바뀔 수 있으나 섞는 횟수를 늘려서 그나마 상쇠시킬 수 있다


In [ ]:
print("Application dataframe shape: ", df.shape)

# bureau.csv 데이터셋 가공
- application data from previous loans that client got from other institutions and that were reported to Credit Bureau
- One row per client's loan in Credit Bureau

In [ ]:
# bureau데이터 사용
# bureau_df = get_bureau(DATA_DIRECTORY, num_rows= num_rows)
# df = pd.merge(df, bureau_df, on='SK_ID_CURR', how='left')
# print("Bureau dataframe shape: ", bureau_df.shape)
# del bureau_df; gc.collect()

# get_bureau 메소드

In [ ]:
""" Process bureau.csv and bureau_balance.csv and return a pandas dataframe. """
bureau = pd.read_csv(os.path.join(path, 'bureau.csv'), nrows= num_rows)

In [ ]:
# Credit duration and credit/account end date difference
bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
# Credit to debt ratio and difference
bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']